In [1]:
from generation import Llama
import torch
import gc
import os
os.environ['RANK'] = '0'  # Example rank, adjust accordingly
os.environ['WORLD_SIZE'] = '1'  # Example world size, adjust accordingly
os.environ['MASTER_ADDR'] = 'localhost'  # Example master address
os.environ['MASTER_PORT'] = '12355'  # Example master port


In [2]:

llama = Llama.build(ckpt_dir= "../llama3/Meta-Llama-3-8B-Instruct/", tokenizer_path="../llama3/Meta-Llama-3-8B-Instruct/tokenizer.model", max_seq_len= 512, max_batch_size = 4, model_parallel_size=1)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:690: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Loaded in 37.20 seconds


In [3]:
cpu = torch.device("cpu")
cuda = torch.device("cuda")

In [4]:
llama.model.to(cuda)
model = llama.model

In [5]:
tokenizer = llama.tokenizer

In [35]:



prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|>
<|start_header_id|>user<|end_header_id|>

{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

def get_prompt(prompt: str):
    return torch.tensor(tokenizer.encode(prompt_template.format(user_prompt = prompt), eos= False, bos = False), dtype= torch.long).unsqueeze(0)




prompt_tokens = get_prompt("Write a poem in 1000 words")


In [52]:
print(tokenizer.decode(prompt_tokens[0].tolist()))


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Write a poem in 1000 words<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [51]:
out_tokens = llama.generate(prompt_tokens.tolist(), max_gen_len= 256, temperature = 0.1)

100%|██████████| 256/256 [00:09<00:00, 26.53it/s]


In [53]:
print(tokenizer.decode(out_tokens[0][0]))

A world of wonder, a world of might,
Where cultures blend and stories take flight,
A tapestry rich, with threads of old,
A journey awaits, for hearts of gold.

In cities bustling, with sounds and sights,
Where ancient ruins whisper secrets at night,
In mountains towering, where eagles soar,
And oceans vast, where dolphins play once more.

In markets vibrant, with colors bright,
Where spices waft, and aromas take flight,
In temples serene, with incense and prayer,
Where souls connect, and hearts are aware.

In forests dark, where myths unfold,
Where creatures roam, and legends are told,
In deserts vast, where stars shine bright,
Where nomads roam, and stories take flight.

In every place, a story's told,
Of people, places, and moments old,
Of struggles, triumphs, and hearts that beat,
Of dreams that soar, and souls that meet.

A world of wonder, a world of might,
Where cultures blend, and stories take flight,
A journey awaits, for hearts of gold,
To explore, to discover, to unfold.

In 

In [39]:
len(out_tokens[0][0])

428

In [47]:
prompt_tokens

tensor([[  198,    27,    91,  7413,  3659,  4424,    91,  1822,    91,  2527,
          8932,   851,    91,    29,  9125,    27,    91,   408,  8932,   851,
            91,  1363,  2675,   527,   264, 11190, 15592, 18328,   369,  5944,
         10631,   323, 19075,    27,    91,    68,   354,   851,    91,   397,
            27,    91,  2527,  8932,   851,    91,    29,   882,    27,    91,
           408,  8932,   851,    91,  1363,  8144,   264, 33894,   304,   220,
          1041,    15,  4339,    27,    91,    68,   354,   851,    91,  1822,
            91,  2527,  8932,   851,    91,    29, 78191,    27,    91,   408,
          8932,   851,    91,  1363]])